# Notebook for Drone Navigation Through Maze

This notebook seeks to be a controller and reporting for CoDrone. It will use its capabilities to and onboard sensors to navigate a maze and report back the metrics from the flight. 

### Tunnels to Navigate

<div>
<img src="Images/tunnel.jpg" width="400"/>
</div>


### Drone Layout

<div>
<img src="Images/DroneLayout.png" width="800"/>
</div>


I will attempt to take advantage of the Front Range Sensor as well as the Optical Flow and and Bottom Range Sensor.

Quick Defitinons:

Front Range Sensor - 

Bottom Range Sensor - 

Optical Flow Sensor - 

#### Controlling the Drone

In [4]:
from codrone_edu.drone import *
import time

ModuleNotFoundError: No module named 'codrone_edu'